# SVM
SVM的优缺点：
> 优点：泛化错误率低，计算开销不大，结果易解释。
> 缺点：对参数调节和核函数的选择较为敏感，原始分类器不做修改仅适用于二分类问题。
> 适用数据类型：数值型和标称型。

SVM的一般流程：
> 1. 收集数据:可以使用任意方法。
> 2. 准备数据:需要数值型数据。
> 3. 分析数据:有助于可视化分隔超平面。
> 4. 训练算法:SVM的大部分时间都源自训练,该过程主要实现两个参数的调优。
> 5. 测试算法:十分简单的计算过程就可以实现。
> 6. 使用算法:几乎所有分类问题都可以使用SVM,值得一提的是,SVM本身是一个二类分类器,对多类问题应用SVM需要对代码做一些修改。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

使用拉格朗日乘子法可以得到SVM最优化问题的对偶问题如下：
$$
\begin{align}
\max_{\alpha}&\sum_{i=1}^{m}\alpha_i-\frac12\sum_{i=1}^m\sum_{j=1}^m\alpha_i\alpha_jy_iy_j\boldsymbol{x}_i^T\boldsymbol{x}_j \tag{1} \\
s.t. &\sum_{i=1}^m\alpha_iy_i=0 \\
&\alpha_i\geq0, i=1,2,...,m.
\end{align}
$$
我们的目标就是求解（1）式在约束条件下的最优解。这是一个二次规划问题。<br>
SMO的基本思路是先固定$\alpha_i$之外的所有参数，然后求$\alpha_i$上的极值。由于存在约束${\sum}_{i=1}^m\alpha_iy_i=0$，若固定$\alpha_i$之外的其他变量，则$\alpha_i$可以由其他变量导出。于是，SMO每次选择两个变量$\alpha_i$和$\alpha_j$，并固定其他参数，这样在初始化后，SMO不断执行以下两个步骤直至收敛：
> 1. 选取一对需要更新的变量$\alpha_i$和$\alpha_j$；
> 2. 固定$\alpha_i$和$\alpha_j$以外的参数，求解（1）式获得更新后的$\alpha_i$和$\alpha_j$。

## 1. 使用简化版SMO处理小规模数据集

In [7]:
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

def selectJrand(i, m):
    """随机选择一个整数
    
    Parameters
    ------------
    i : 第一个alpha的下标
    m : 所有alpha的数目
    
    Returns
    ------------
    j : 一个不为i的随机数，在0～m之间
    """
    j = i
    while (j == i):
        j = int(np.random.uniform(0, m))
    return j

def clipAlpha(aj, H, L):
    """调整aj的值，使L<=aj<=H"""
    
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

smoSimple，简化版的SMO算法函数：

In [10]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    """SMO算法简化版
    
    Parameters
    -----------
    dataMatIn : 输入数据集
    classLabel : 类别标签
    C : 松弛变量，允许少量数据点被误分类，控制最大化间隔。
    toler : 容错率
    maxIter : 退出最大的循环次数
    
    Returns
    -----------
    b : 模型的截距值
    alphas : 拉格朗日乘子
    """
    
    dataMatrix = np.mat(dataMatIn); labelMat = np.mat(classLabels).T
    # 参数初始化
    b = 0; m, n = dataMatrix.shape
    alphas = np.mat(np.zeros((m, 1)))
    iter = 0
    while (iter < maxIter):
        # alphaPairsChanged记录alpha是否已经进行优化
        alphaPairsChanged = 0
        for i in range(m):
            # 预测的输出，y = w^Tx[i] + b;其中w = Σ(1~n) a[n]*lable[n]*x[n]
            fXi = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T)) + b
            # Ei为误差
            Ei = fXi - float(labelMat[i])
            
            # KKT条件
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or \
                ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                # 满足条件，选取非i的点进行优化比较
                j = selectJrand(i, m)
                # 预测j的结果
                fXj = float(np.multiply(alphas, labelMat).T * \
                      (dataMatrix * dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                
                # L和H是alpha[j]的下界和上界
                # 详情见《统计学习方法》
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H: print("L == H"); continue
                    
                # eta是alpha[j]的最佳修改量，如果eta==0，则本次循环并没有对alpha优化，所以退出
                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T - \
                      dataMatrix[i, :] * dataMatrix[i, :].T - \
                      dataMatrix[j, :] * dataMatrix[j, :].T
                if eta >= 0: print("eta >= 0"); continue
                # 计算得到一个新的alpha[j]值，然后对alpha[j]进行剪辑，使它约束在L和H之间
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                # 如果alpha[j]的该变量太小，则认为没有被优化，就退出循环。
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough"); continue
                    
                # 根据alpha[j]求出alpha[i]的值，虽然改变带下一样，但是方向相反
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                # 更新b的值
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * \
                     dataMatrix[i, :] * dataMatrix[i, :].T - \
                     labelMat[j] * (alphas[j] - alphaJold) * \
                     dataMatrix[i, :] * dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * \
                     dataMatrix[i, :] * dataMatrix[j, :].T - \
                     labelMat[j] * (alphas[j] - alphaJold) * \
                     dataMatrix[j, :] * dataMatrix[j, :].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2) / 2.0
                # 设置alphaPairsChanged加1
                alphaPairsChanged += 1
                print("iter: %d  i: %d, pairs changed %d" % (iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print("iteration number: %d" % iter)
    return b, alphas

In [11]:
dataArr, labelArr = loadDataSet('testSet.txt')
dataArr[:10], labelArr[:10]

([[3.542485, 1.977398],
  [3.018896, 2.556416],
  [7.55151, -1.58003],
  [2.114999, -0.004466],
  [8.127113, 1.274372],
  [7.108772, -0.986906],
  [8.610639, 2.046708],
  [2.326297, 0.265213],
  [3.634009, 1.730537],
  [0.341367, -0.894998]],
 [-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0])

In [12]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

L == H
iter: 0  i: 1, pairs changed 1
L == H
L == H
iter: 0  i: 8, pairs changed 2
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
L == H
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
L == H
iter: 0  i: 26, pairs changed 1
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 97, pairs changed 2
iteration number: 0
j not moving enough
L == H
L == H
L == H
j not moving enough
iter: 0  i: 30, pairs changed 1
L == H
j not moving enough
j not moving enough
L == H
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
iter: 0  i: 52, pairs changed 1
L == H
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
iter: 0  i: 8, pairs changed 1
L == H
j not moving enough
j not moving enough
j not moving enou

j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
iter: 3  i: 23, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
iter: 2  i: 30, pairs changed 1
iter: 2  i: 52, pairs changed 2
j not moving enough
iter

iteration number: 8
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
iter: 10  i: 55, pairs changed 1
iteration number: 0
L == H
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
L == H
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
L == H
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
L == H
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
L == H
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration n

In [13]:
b

matrix([[-3.72504509]])

In [14]:
alphas[alphas>0]

matrix([[0.17111467, 0.09207197, 0.0977385 , 0.36092514]])

## 2. 使用完整版的Platt SMO算法加速优化

在这两个版本中实现$\alpha$的更改和代数运算的优化环节一模一样，不同的是$\alpha$的选择方式，完整版的$\alpha$采用了一种启发式的方法。选择完第一个alpha的值后，会采用最大化步长的方式获得第二个$\alpha$值。<br>
由前面可知，$\alpha_2$的选择是依赖于$|E_1 - E_2|$的，为了加快计算速度，可以选择对应$|E_1 - E_2|$最大的。因为$\alpha_1$已定，$E_1$也确定了，如果$E_1$是正的，那么选择最小的$E_i$作为$E_2$，如果$E_1$是负的，那么选择最大的$E_i$作为$E_2$，为了节省计算时间，可以将所有的$E_i$保存在一个列表中。<br>
并且为了减少代码量，我们在这里加入核函数。

In [38]:
# 用于对代码进行清理的数据结构和对E进行缓存的辅助函数
class optStruct(object):
    """
    建立的数据结构来保存所有的重要值
    """
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        # 误差缓存，第一列给出的是eCache是否有效的标志位，第二列给出的是实际的E值。
        self.eCache = np.mat(np.zeros((self.m, 2)))
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)
        
def calcEk(oS, k):
    """calcEk（求 Ek误差：预测值-真实值的差）
    该过程在完整版的SMO算法中陪出现次数较多，因此将其单独作为一个方法
        
    Parameters
    -----------
    oS : optStruct对象
    k : 具体的某一行
        
    Returns
    -----------
    Ek : 预测结果与真实结果比对，计算误差Ek
    """
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k, :].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek
    
def selectJ(i, oS, Ei):
    """selectJ（返回最优的j和Ej）
    内循环的启发式方法
        
    Parameters
    -----------
    i : 具体的第i一行
    oS : optStruct对象
    Ei : 预测结果与真实结果比对，计算误差Ei
        
    Returns
    -----------
    j : 随机选出的第j一行
    Ej : 预测结果与真实结果比对，计算误差Ej
    """
    maxK = -1; maxDeltaE = 0; Ej = 0
    # 首先将输入值Ei在缓存中设置成为有效的。这里的有效意味着它已经计算好了。
    oS.eCache[i] = [1, Ei]
    # 存储可用的所有Ei的值
    validEcacheList = np.nonzero(oS.eCache[:, 0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i: continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej
    
def updateEk(oS, k):
    """updateEk（计算误差值并存入缓存中。）在对alpha值进行优化之后会用到这个值。
    
    Parameters
    -----------
    oS : optStruct对象
    k : 某一列的行号
    """
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]
    
def kernelTrans(X, A, kTup):
    """
    核转换函数
    
    Parameters
    -----------
    X : dataMatIn数据集
    A : dataMatIn的第i行数据
    kTup : 核函数信息
    
    Returns
    -----------
    K : 核函数计算结果
    """
    m, n = np.shape(X)
    K = np.mat(np.zeros((m, 1)))
    if kTup[0] == 'lin': K = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow * deltaRow.T
        K = np.exp(K / (-1 * kTup[1] ** 2))
    else:
        raise NameError("Houston We Have a Problem -- That Kernel is not recognized")
    return K

In [39]:
def innerL(i, oS):
    """innerL
    内循环代码
    
    Parameters
    -----------
    i : 具体的某一行
    oS : optStruct对象
    
    Returns
    -----------
    0 : 找不到最优的值
    1 : 找到了最优的值，并且oS.Cache到缓存中
    """
    Ei = calcEk(oS, i)
    
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        
        # L和H用于将alphas[j]调整到0-C之间。如果L==H，就不做任何改变，直接return 0
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H: return 0
        
        # eta是alphas[j]的最优修改量，如果eta==0，需要退出for循环的当前迭代过程
        # 参考《统计学习方法》李航-P125~P128<序列最小最优化算法>
        # eta = 2.0 * oS.X[i, :] * oS.X[j, :].T - oS.X[i, :] * oS.X[i, :].T - oS.X[j, :] * oS.X[j, :].T
        eta = 2.0 * oS.K[i, j] - oS.K[i, i] - oS.K[j, j]   # change for kernel
        if eta >= 0: print("eta >= 0"); return 0
        
        # 计算出一个新的alphas[j]值，并进行剪辑，更新误差缓存
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)
        
        # 检查alpha[j]是否只是轻微的改变，如果是的话，就退出for循环。
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print("j not moving enough"); return 0
        
        # 然后alphas[i]和alphas[j]同样进行改变，虽然改变的大小一样，但是改变的方向正好相反，并更新误差缓存
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS, i)
        
#         b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * \
#              oS.X[i, :] * oS.X[i, :].T - oS.labelMat[j] * \
#              (oS.alphas[j] - alphaJold) * oS.X[i, :] * oS.X[j, :].T
#         b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * \
#              oS.X[i, :] * oS.X[j, :].T - oS.labelMat[j] * \
#              (oS.alphas[j] - alphaJold) * oS.X[j, :] * oS.X[j, :].T

        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * \
             oS.K[i, i] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[i, j]
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * \
             oS.K[i, j] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[j, j]
        
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0
    
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    """
    完整SMO算法外循环，与smoSimple有些类似，但这里的循环退出条件更多一些
    Parameters
    -----------
        dataMatIn : 数据集
        classLabels : 类别标签
        C : 松弛变量(常量值)，允许有些数据点可以处于分隔面的错误一侧。
            控制最大化间隔和保证大部分的函数间隔小于1.0这两个目标的权重。
            可以通过调节该参数达到不同的结果。
        toler : 容错率
        maxIter : 退出前最大的循环次数
        kTup : 包含核函数信息的元组
    
    Returns
    -----------
        b : 模型的截距值
        alphas : 拉格朗日乘子
    """
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).T, C, toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    # 循环遍历：循环maxIter次 并且 （alphaPairsChanged存在可以改变 or 所有行遍历一遍）
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i, oS)
            print("FullSet, iter: %d i: %d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False
        elif (alphaPairsChanged == 0): entireSet = True
        print("iteration number: %d" % iter)
    return oS.b, oS.alphas

In [40]:
# 计算权重参数的值用于分类
def calcWs(alphas, dataArr, classLabels):
    X = np.mat(dataArr); labelMat = np.mat(classLabels).T
    m, n = X.shape
    w = np.zeros((n, 1))
    for i in range(m):
        w += np.multiply(alphasp[i] * labelMat[i], X[i, :].T)
    return w

构建一个径向基核函数用于分类：

In [47]:
def testRbf(k1 = 1.3):
    dataArr, labelArr = loadDataSet('testSetRBF.txt')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))
    datMat = np.mat(dataArr); labelMat = np.mat(labelArr).T
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print("There are %d Support Vectors" % np.shape(sVs)[0])
    m, n = datMat.shape
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("The training error rate is: %f" % (float(errorCount) / m))
    
    dataArr, labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).T
    m, n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the test error rate is: %f" % (float(errorCount) / m))

In [48]:
testRbf()

j not moving enough
j not moving enough
j not moving enough
FullSet, iter: 0 i: 99, pairs changed 67
iteration number: 1
non-bound, iter: 1 i:0, pairs changed 1
non-bound, iter: 1 i:5, pairs changed 2
non-bound, iter: 1 i:8, pairs changed 3
non-bound, iter: 1 i:9, pairs changed 4
non-bound, iter: 1 i:10, pairs changed 5
non-bound, iter: 1 i:12, pairs changed 6
non-bound, iter: 1 i:13, pairs changed 7
non-bound, iter: 1 i:14, pairs changed 8
non-bound, iter: 1 i:15, pairs changed 9
non-bound, iter: 1 i:16, pairs changed 10
non-bound, iter: 1 i:18, pairs changed 11
non-bound, iter: 1 i:19, pairs changed 12
non-bound, iter: 1 i:22, pairs changed 13
non-bound, iter: 1 i:23, pairs changed 14
non-bound, iter: 1 i:26, pairs changed 15
non-bound, iter: 1 i:28, pairs changed 16
non-bound, iter: 1 i:29, pairs changed 17
non-bound, iter: 1 i:33, pairs changed 18
non-bound, iter: 1 i:34, pairs changed 19
non-bound, iter: 1 i:35, pairs changed 20
non-bound, iter: 1 i:36, pairs changed 21
non-bound,

j not moving enough
non-bound, iter: 4 i:16, pairs changed 8
j not moving enough
non-bound, iter: 4 i:18, pairs changed 8
j not moving enough
non-bound, iter: 4 i:19, pairs changed 8
j not moving enough
non-bound, iter: 4 i:22, pairs changed 8
j not moving enough
non-bound, iter: 4 i:23, pairs changed 8
j not moving enough
non-bound, iter: 4 i:26, pairs changed 8
j not moving enough
non-bound, iter: 4 i:28, pairs changed 8
j not moving enough
non-bound, iter: 4 i:29, pairs changed 8
j not moving enough
non-bound, iter: 4 i:33, pairs changed 8
j not moving enough
non-bound, iter: 4 i:34, pairs changed 8
j not moving enough
non-bound, iter: 4 i:35, pairs changed 8
non-bound, iter: 4 i:36, pairs changed 9
non-bound, iter: 4 i:39, pairs changed 10
j not moving enough
non-bound, iter: 4 i:40, pairs changed 10
j not moving enough
non-bound, iter: 4 i:43, pairs changed 10
j not moving enough
non-bound, iter: 4 i:44, pairs changed 10
j not moving enough
non-bound, iter: 4 i:45, pairs changed 1

non-bound, iter: 6 i:96, pairs changed 20
non-bound, iter: 6 i:98, pairs changed 21
non-bound, iter: 6 i:99, pairs changed 22
iteration number: 7
non-bound, iter: 7 i:0, pairs changed 1
j not moving enough
non-bound, iter: 7 i:5, pairs changed 1
non-bound, iter: 7 i:8, pairs changed 2
non-bound, iter: 7 i:9, pairs changed 3
non-bound, iter: 7 i:10, pairs changed 4
non-bound, iter: 7 i:12, pairs changed 5
non-bound, iter: 7 i:13, pairs changed 6
non-bound, iter: 7 i:14, pairs changed 7
non-bound, iter: 7 i:15, pairs changed 8
non-bound, iter: 7 i:16, pairs changed 9
non-bound, iter: 7 i:18, pairs changed 10
non-bound, iter: 7 i:19, pairs changed 11
j not moving enough
non-bound, iter: 7 i:22, pairs changed 11
j not moving enough
non-bound, iter: 7 i:23, pairs changed 11
j not moving enough
non-bound, iter: 7 i:26, pairs changed 11
j not moving enough
non-bound, iter: 7 i:28, pairs changed 11
j not moving enough
non-bound, iter: 7 i:29, pairs changed 11
j not moving enough
non-bound, ite

non-bound, iter: 9 i:89, pairs changed 10
non-bound, iter: 9 i:90, pairs changed 11
j not moving enough
non-bound, iter: 9 i:93, pairs changed 11
j not moving enough
non-bound, iter: 9 i:94, pairs changed 11
non-bound, iter: 9 i:96, pairs changed 12
non-bound, iter: 9 i:98, pairs changed 13
non-bound, iter: 9 i:99, pairs changed 14
iteration number: 10
j not moving enough
non-bound, iter: 10 i:0, pairs changed 0
non-bound, iter: 10 i:8, pairs changed 1
non-bound, iter: 10 i:9, pairs changed 2
j not moving enough
non-bound, iter: 10 i:10, pairs changed 2
j not moving enough
non-bound, iter: 10 i:12, pairs changed 2
j not moving enough
non-bound, iter: 10 i:13, pairs changed 2
j not moving enough
non-bound, iter: 10 i:14, pairs changed 2
non-bound, iter: 10 i:15, pairs changed 3
j not moving enough
non-bound, iter: 10 i:16, pairs changed 3
j not moving enough
non-bound, iter: 10 i:18, pairs changed 3
j not moving enough
non-bound, iter: 10 i:19, pairs changed 3
j not moving enough
non-bo

non-bound, iter: 12 i:88, pairs changed 8
non-bound, iter: 12 i:90, pairs changed 9
j not moving enough
non-bound, iter: 12 i:93, pairs changed 9
j not moving enough
non-bound, iter: 12 i:94, pairs changed 9
non-bound, iter: 12 i:96, pairs changed 10
non-bound, iter: 12 i:98, pairs changed 11
non-bound, iter: 12 i:99, pairs changed 12
iteration number: 13
j not moving enough
non-bound, iter: 13 i:0, pairs changed 0
non-bound, iter: 13 i:8, pairs changed 1
non-bound, iter: 13 i:9, pairs changed 2
j not moving enough
non-bound, iter: 13 i:10, pairs changed 2
j not moving enough
non-bound, iter: 13 i:12, pairs changed 2
j not moving enough
non-bound, iter: 13 i:13, pairs changed 2
j not moving enough
non-bound, iter: 13 i:14, pairs changed 2
non-bound, iter: 13 i:15, pairs changed 3
j not moving enough
non-bound, iter: 13 i:16, pairs changed 3
j not moving enough
non-bound, iter: 13 i:18, pairs changed 3
j not moving enough
non-bound, iter: 13 i:19, pairs changed 3
j not moving enough
non

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
FullSet, iter: 15 i: 99, pairs changed 0
iteration number: 16
There are 66 Support Vectors
The training error rate is: 0.560000
the test error rate is: 0.490000
